# **Intercomparison Demo**

**Author:** Adam Yang (ayang115@umd.edu) and Hannah Kerner (hkerner@asu.edu)

**Description** Demo for Intercomparison Script: extracts manually labeled test points from publicly-available land cover maps (Harvest, Copernicus, ESA, GLAD, etc) and evaluates performance metrics for each map.

In [ ]:
# !earthengine authenticate

In [ ]:
import ee
ee.Initialize()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ee
from shapely import wkt

import sys
sys.path.append("..")
from src.compare_covermaps import *

In [ ]:
maps = [v for v in TARGETS.values()]
countries = ["Togo", "Kenya", "Malawi", "Tanzania", "Mali", "Namibia", "Rwanda", "Uganda", "Zambia", "Senegal"]

In [ ]:
covermap_test = TestCovermaps(test_countries=countries, covermaps=maps)

In [ ]:
# Load the test data points
test_pts = covermap_test.get_test_points()

# Alternatively, read from existing file
# test_pts = pd.read_csv("../../intercomparison-data/extracted/test/tgo_ken_tza_mlw_test.csv").drop("Unnamed: 0", axis=1)
# test_pts["geometry"] = test_pts["geometry"].apply(wkt.loads)
# test_pts = gpd.GeoDataFrame(test_pts, crs="epsg:4326")

# Print a random sample of the dataframe
test_pts.sample(10)

In [ ]:
# Notes about warnings:
# Number of sampled points could be more if there are overlapping images in imagecollection
# could only be less if there was missing data in the map?
extracted = covermap_test.extract_covermaps(test_pts)

In [ ]:
# Add ensemble classifications (mode = voting classifier)
comp_maps = ['copernicus', 'worldcover', 'glad', 'asap', 'dynamicworld', 'gfsad-gcep',
             'digital-earth-africa', 'esa-cci-africa', 'globcover-v23', 'esri-lulc', 'nabil-etal-2021']

for country in countries:
    ensemble_subset = extracted[country][comp_maps].mode(axis='columns')
    extracted[country]['ensemble-subset'] = ensemble_subset
    
covermap_test.sampled_maps = extracted
covermap_test.covermap_titles = covermap_test.covermap_titles + ['ensemble-subset']

In [ ]:
_ = covermap_test.evaluate()
results = pd.concat(covermap_test.results)
results

In [ ]:
results.to_csv("../notebooks/intercomparison-results.csv")